# Create HS with CC Machine Learning Dataset

This notebook reads in the merged data file located [here](https://github.com/BrownRegaSterlingHeinen/PostsecondaryAttainment/blob/master/RawDatasets/mergedData_geoHSCC.xlsx) as input data. A lot of the code used in this notebook is from Dr. Jake Drew's repository for North Carolina public school data which can be viewed [here](https://github.com/jakemdrew/EducationDataNC/tree/master/2017/Machine%20Learning%20Datasets). This creates a dataset that is preprocessed for Machine Learning by going through the following transformations:

- Columns that have the same value in every single row are deleted.
- Nominal columns that have a unique value in every single row (all values are different) are deleted.
- Empty columns (all values are NA or NULL) are deleted.
- Numeric columns with more than the percentage of missing values specified by the missingThreshold parameter are deleted.
- Remaining numeric columns with missing values are imputed / populated with the mean of the column.  
- Categorical / text based columns with > uniqueThreshold unique values are deleted.
- Duplicated or highly similar columns with > 95% correlation are delelted.

In [47]:
import pandas as pd
import numpy as np

# Load in merged high school data
url = 'https://github.com/BrownRegaSterlingHeinen/PostsecondaryAttainment/blob/master/2016/NCCC%20Datasets/mergedData_geoHSCC.xlsx?raw=true'
NCCCData = pd.read_excel(url)

In [48]:
# View info for data
NCCCData.info()
NCCCData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 2008 entries, unit_code to $200,000 or more Number of Returns
dtypes: float64(1569), int64(106), object(333)
memory usage: 7.0+ MB


,unit_code,szip_ad_x,Lea_Name_x,School_Name_x,State_Name_x,Latitude_x,Longitude_x,Min_Distance_CC_miles,Min_CC_ID,Min_CC_Latitude,...,Tax due at time of filing [11]_Number of returns,Tax due at time of filing [11]_Amount,Overpayments refunded [12]_Number of returns,Overpayments refunded [12]_Amount,"$1 under $25,000 Number of Returns","$25,000 under $50,000 Number of Returns","$50,000 under $75,000 Number of Returns","$75,000 under $100,000 Number of Returns","$100,000 under $200,000 Number of Returns","$200,000 or more Number of Returns"
0,10303,27253,Alamance-Burlington Schools,Alamance-Burlington Middle/Early College,State Of North Carolina,36.06,-79.36,0.236451,199786,36.06358,...,13290.0,48075.0,58020.0,150599.0,28880.0,19790.0,10120.0,6220.0,7190.0,1500.0
1,10324,27302,Alamance-Burlington Schools,Eastern Alamance High,State Of North Carolina,36.11,-79.30,3.959695,199786,36.06358,...,13290.0,48075.0,58020.0,150599.0,28880.0,19790.0,10120.0,6220.0,7190.0,1500.0
2,10348,27253,Alamance-Burlington Schools,Graham High,State Of North Carolina,36.08,-79.38,1.454274,199786,36.06358,...,13290.0,48075.0,58020.0,150599.0,28880.0,19790.0,10120.0,6220.0,7190.0,1500.0
3,10360,27217,Alamance-Burlington Schools,Hugh M Cummings High,State Of North Carolina,36.09,-79.40,2.583810,199786,36.06358,...,13290.0,48075.0,58020.0,150599.0,28880.0,19790.0,10120.0,6220.0,7190.0,1500.0
4,10388,27253,Alamance-Burlington Schools,Southern High,State Of North Carolina,36.01,-79.41,4.090248,199786,36.06358,...,13290.0,48075.0,58020.0,150599.0,28880.0,19790.0,10120.0,6220.0,7190.0,1500.0


In [49]:
# Missing Data Threshold (Per Column)
missingThreshold = 0.60

# Unique Value Threshold (Per Column)
# Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
# (each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

## Prepare Consolidated Dataset for Machine Learning
Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.

We can see that there are many columns that are meant to be floats that are listed as objects. We need to iteratively go through all of the columns and change these values.

In [50]:
for column in range(1279, len(NCCCData.columns)): #loops through every column in the data set
    if NCCCData.iloc[:,column].isnull().sum() == 0: #these are the string columns that do not need to be changed
        print("Column ", column , "stayed the same")
        # If a column contains '-', it is not reading correctly, the following code deals with those columns
        for value in range(len(NCCCData)):
            if '%' in str(NCCCData.iloc[value,column]):
                NCCCData.iloc[value,column] = float(NCCCData.iloc[value,column].strip("%"))/100
                NCCCData.iloc[:,column] = pd.to_numeric(NCCCData.iloc[:,column], errors='ignore')
            elif NCCCData.iloc[value,column] == "-":
                NCCCData.iloc[value, column] = None
                NCCCData.iloc[:,column] = pd.to_numeric(NCCCData.iloc[:,column], errors='ignore')
    elif NCCCData.iloc[:,column].dtype == "object": #if the column is of type object (percentages)
        for value in range(len(NCCCData)):
            if pd.isnull(NCCCData.iloc[value,column]):
                continue
            elif "%" in NCCCData.iloc[value,column]: # one of the issues is percentages
                if ' ' in NCCCData.iloc[value,column]:
                    NCCCData.iloc[value,column] = NCCCData.iloc[value,column].split((' ', 1)[0])[0]
                NCCCData.iloc[value,column] = float(NCCCData.iloc[value,column].strip("%"))/100
            elif "," in NCCCData.iloc[value, column]: # another issues is commas
                NCCCData.iloc[value, column] = float(NCCCData.iloc[value, column].replace(",", ""))
            elif NCCCData.iloc[value,column] == "*":
                NCCCData.iloc[value, column] = None
            #elif NCCCData.iloc[value,column] == "-":
             #   NCCCData.iloc[value, column] = NCCCData.iloc[value, column].replace("-", "")
            else:
                continue
        print("Column ", column, "was changed to type numeric")
        NCCCData.iloc[:,column] = pd.to_numeric(NCCCData.iloc[:,column])
    else:
        print("Column", column, "stayed the same")

Column  1279 stayed the same
Column  1280 stayed the same
Column  1281 stayed the same
Column  1282 stayed the same
Column  1283 stayed the same
Column  1284 stayed the same
Column  1285 stayed the same
Column  1286 stayed the same
Column 1287 stayed the same
Column  1288 stayed the same
Column  1289 was changed to type numeric
Column 1290 stayed the same
Column 1291 stayed the same
Column  1292 was changed to type numeric
Column 1293 stayed the same
Column  1294 was changed to type numeric
Column  1295 was changed to type numeric
Column  1296 was changed to type numeric
Column  1297 was changed to type numeric
Column  1298 was changed to type numeric
Column  1299 was changed to type numeric
Column 1300 stayed the same
Column  1301 was changed to type numeric
Column 1302 stayed the same
Column  1303 was changed to type numeric
Column 1304 stayed the same
Column  1305 was changed to type numeric
Column 1306 stayed the same
Column  1307 was changed to type numeric
Column 1308 stayed the 

Column  1502 was changed to type numeric
Column  1503 was changed to type numeric
Column  1504 was changed to type numeric
Column 1505 stayed the same
Column  1506 was changed to type numeric
Column  1507 was changed to type numeric
Column  1508 was changed to type numeric
Column  1509 was changed to type numeric
Column 1510 stayed the same
Column  1511 was changed to type numeric
Column  1512 was changed to type numeric
Column  1513 was changed to type numeric
Column  1514 was changed to type numeric
Column 1515 stayed the same
Column  1516 was changed to type numeric
Column  1517 was changed to type numeric
Column  1518 was changed to type numeric
Column  1519 was changed to type numeric
Column 1520 stayed the same
Column  1521 was changed to type numeric
Column  1522 was changed to type numeric
Column  1523 was changed to type numeric
Column  1524 was changed to type numeric
Column 1525 stayed the same
Column  1526 was changed to type numeric
Column  1527 was changed to type numeric

Column  1746 stayed the same
Column  1747 stayed the same
Column  1748 stayed the same
Column  1749 stayed the same
Column  1750 stayed the same
Column  1751 stayed the same
Column  1752 stayed the same
Column  1753 stayed the same
Column  1754 stayed the same
Column  1755 stayed the same
Column  1756 stayed the same
Column  1757 stayed the same
Column  1758 stayed the same
Column  1759 stayed the same
Column  1760 stayed the same
Column  1761 stayed the same
Column  1762 stayed the same
Column  1763 stayed the same
Column  1764 stayed the same
Column  1765 stayed the same
Column  1766 stayed the same
Column  1767 stayed the same
Column  1768 stayed the same
Column 1769 stayed the same
Column 1770 stayed the same
Column 1771 stayed the same
Column 1772 stayed the same
Column 1773 stayed the same
Column 1774 stayed the same
Column 1775 stayed the same
Column 1776 stayed the same
Column 1777 stayed the same
Column 1778 stayed the same
Column 1779 stayed the same
Column 1780 stayed the sa

### Impute any Remaining Missing Values with Mean Value
Missing values will be imputed with the mean of the column.

In [51]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be Imputed with the Mean!*********')

#Check for Missing values 
missing_values = NCCCData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values


*********The Remaining Missing Values Below will be Imputed with the Mean!*********


,Variable Name,Number Missing Values
25,super_nm,454
27,url_ad,15
31,cover_letter_ad,196
34,title1_type_cd,424
35,clp_ind,400
36,focus_clp_ind,402
37,summer_program_ind,285
38,asm_no_spg_ind,454
39,no_data_spg_ind,454
43,esea_status,438


In [52]:
#Replace all remaining NaN with mean of column
NCCCData = NCCCData.fillna(NCCCData.mean())

#Check for Missing values after final imputation 
missing_values = NCCCData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values
25,super_nm,454
27,url_ad,15
31,cover_letter_ad,196
34,title1_type_cd,424
35,clp_ind,400
36,focus_clp_ind,402
37,summer_program_ind,285
38,asm_no_spg_ind,454
39,no_data_spg_ind,454
43,esea_status,438


The remaining columns with missing values are completely empty, so these will be deleted.

In [53]:
NCCCData = NCCCData.dropna(axis='columns', how='all')

### Remove Columns with Problematic Data
Here we remove entire columns that could cause problems during machine learning. The following operations are performed:

- Remove any columns that have the same value in every single row.
- Remove any columns that have a unique value in every single row (all values are different).
- Remove empty columns (all values are NA or NULL).

In [54]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = NCCCData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
NCCCData = NCCCData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1172 entries, unit_code to $200,000 or more Number of Returns
dtypes: float64(1036), int64(94), object(42)
memory usage: 4.1+ MB

Columns Deleted:  303


In [55]:
#Remove any fields that have unique values in every row
NCCCDataRecordCt = NCCCData.shape[0]
UniqueValueCounts = NCCCData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == NCCCDataRecordCt].index
NCCCData = NCCCData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1170 entries, szip_ad_x to $200,000 or more Number of Returns
dtypes: float64(1036), int64(93), object(41)
memory usage: 4.1+ MB

Columns Deleted:  2


In [56]:
#Remove any empty fields (null values in every row)
NCCCDataRecordCt = NCCCData.shape[0]
NullValueCounts = NCCCData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == NCCCDataRecordCt].index
NCCCData = NCCCData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1170 entries, szip_ad_x to $200,000 or more Number of Returns
dtypes: float64(1036), int64(93), object(41)
memory usage: 4.1+ MB

Columns Deleted:  0


### Handle Other Missing Values Types
- Here we eliminate any numeric and continous columns with more than the percentage of missing values specified by the missingThreshold parameter.
- All remaining missing values are populated with 0.

In [57]:
#Isolate continuous and categorical data types
#These are indexers into the NCCCData dataframe and may be used similar to the schoolData dataframe 
CCD_boolean = NCCCData.loc[:, (NCCCData.dtypes == bool) ]
CCD_nominal = NCCCData.loc[:, (NCCCData.dtypes == object)]
CCD_continuous = NCCCData.loc[:, (NCCCData.dtypes != bool) & (NCCCData.dtypes != object)]
print("Boolean Columns: ", CCD_boolean.shape[1])
print("Nominal Columns: ", CCD_nominal.shape[1])
print("Continuous Columns: ", CCD_continuous.shape[1])
print("Columns Accounted for: ", CCD_nominal.shape[1] + CCD_continuous.shape[1] + CCD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  41
Continuous Columns:  1129
Columns Accounted for:  1170


In [58]:
#Eliminate nominal columns with more than missingThreshold percentage of missing values
NCCCDataRecordCt = CCD_nominal.shape[0]
missingValueLimit = NCCCDataRecordCt * missingThreshold
NullValueCounts = CCD_nominal.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
NCCCData = NCCCData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1161 entries, szip_ad_x to $200,000 or more Number of Returns
dtypes: float64(1036), int64(93), object(32)
memory usage: 4.0+ MB

Columns Deleted:  9


In [59]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
NCCCDataRecordCt = CCD_continuous.shape[0]
missingValueLimit = NCCCDataRecordCt * missingThreshold
NullValueCounts = CCD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
NCCCData = NCCCData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1161 entries, szip_ad_x to $200,000 or more Number of Returns
dtypes: float64(1036), int64(93), object(32)
memory usage: 4.0+ MB

Columns Deleted:  0


### Categorical Variables
Any categorical variables that have greater than the set number of unique values, the uniqueThreshold, in each column will be deleted. Other categorical variables will be one-hot encoded.

In [60]:
#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
CCD_nominal = NCCCData.loc[:, (NCCCData.dtypes == object)]
oneHotUniqueValueCounts = NCCCData[CCD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
NCCCData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
NCCCData.info(verbose=False)
print('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1142 entries, State_Name_x to $200,000 or more Number of Returns
dtypes: float64(1036), int64(93), object(13)
memory usage: 4.0+ MB

Columns Deleted:  19


In [61]:
#Isolate remaining categorical variables
begColumnCt = len(NCCCData.columns)
CCD_nominal = NCCCData.loc[:, (NCCCData.dtypes == object)]

#one hot encode categorical variables
if (len(CCD_nominal.columns) != 0):
    onehotcols = pd.get_dummies(CCD_nominal, drop_first=True)
    #Determine change in column count
    NCCCData = pd.concat([NCCCData, onehotcols], axis=1)
    endColumnCt = len(NCCCData.columns)
    columnsAdded = endColumnCt - begColumnCt

    #Review dataset contents one hot high unique value drops
    print('Columns To One-Hot Encode: ', len(CCD_nominal.columns))
    print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
    NCCCData.info(verbose=False)
    print('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)
else:
    print("No categorical variables to one-hot encode.")

Columns To One-Hot Encode:  13

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1187 entries, State_Name_x to grades_1_to_1_access_9:10:12
dtypes: float64(1036), int64(93), object(13), uint8(45)
memory usage: 4.0+ MB

New Columns Created Via One-Hot Encoding:  45


### Identify and Remove Highly Correlated Features
Find and remove any columns / features that are > 95% correlated

In [63]:
# calculate the correlation matrix
corr_matrix  = NCCCData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [64]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print('There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  14092  field correlations > 95%.


,Field1,Field2,Correlation
0,Min_CC_Latitude,Latitude_x,0.977486
1,Latitude_y,Latitude_x,0.977486
2,Min_CC_Longitude,Longitude_x,0.995845
3,Min_CC_Longitude.1,Longitude_x,0.982949
4,Longitude_y,Longitude_x,0.995845
5,UnitId_x,Min_CC_ID,1.000000
6,UnitId_x.1,Min_CC_ID,1.000000
7,UnitId_x.2,Min_CC_ID,1.000000
8,UnitId_y,Min_CC_ID,1.000000
9,UnitId_y.1,Min_CC_ID,1.000000


In [65]:
print('Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
to_drop

Dropping the following  509  highly correlated fields.


['Min_CC_Latitude',
 'Min_CC_Longitude',
 'Min_CC_Longitude.1',
 'Overall Achievement Score',
 'ACTCompositeScore_UNCMin_All',
 'ACTEnglish_ACTBenchmark_All',
 'ACTReading_ACTBenchmark_All',
 'ACTSubtests_BenchmarksMet_All',
 'ACTWorkKeys_SilverPlus_All',
 'EOCBiology_CACR_All',
 'EOCBiology_GLP_All',
 'EOCEnglish2_CACR_All',
 'EOCEnglish2_GLP_All',
 'EOCMathI_CACR_All',
 'EOCMathI_GLP_All',
 'EOCSubjects_GLP_All',
 'EOG/EOCSubjects_CACR_All',
 'EOG/EOCSubjects_GLP_All',
 'GraduationRate_4yr_All',
 'ACTEnglish_ACTBenchmark_Female',
 'ACTMath_ACTBenchmark_Female',
 'ACTReading_ACTBenchmark_Female',
 'ACTScience_ACTBenchmark_Female',
 'ACTSubtests_BenchmarksMet_Female',
 'ACTWriting_ACTBenchmark_Female',
 'EOCBiology_GLP_Female',
 'EOCEnglish2_CACR_Female',
 'EOCEnglish2_GLP_Female',
 'EOCMathI_CACR_Female',
 'EOCMathI_GLP_Female',
 'EOCSubjects_CACR_Female',
 'EOCSubjects_GLP_Female',
 'EOG/EOCSubjects_CACR_Female',
 'EOG/EOCSubjects_GLP_Female',
 'ACTSubtests_BenchmarksMet_Male',
 'EOC

In [66]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
NCCCData.info(verbose=False)

# Drop the highly correlated features from our training data 
NCCCData = NCCCData.drop(to_drop, axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
NCCCData.info(verbose=False)

#Save the final dataset to a .csv file
NCCCData.to_csv('NCCCData_HSCCwithGeo' + '_ML.csv', sep=',', index=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 1187 entries, State_Name_x to grades_1_to_1_access_9:10:12
dtypes: float64(1036), int64(93), object(13), uint8(45)
memory usage: 4.0+ MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Columns: 678 entries, State_Name_x to grades_1_to_1_access_9:10:12
dtypes: float64(582), int64(43), object(13), uint8(40)
memory usage: 2.2+ MB


In [67]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
NCCCData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 678 columns):
State_Name_x                                                    object
Latitude_x                                                      float64
Longitude_x                                                     float64
Min_Distance_CC_miles                                           float64
Min_CC_ID                                                       int64
2_Min_Distance_CC_miles                                         float64
2_Min_CC_ID                                                     int64
Min_CC_Latitude.1                                               float64
szip_ad_y                                                       int64
category_cd                                                     object
grade_range_cd                                                  object
calendar_type_txt     